# 'A model of Quantum Gravity on a Noisy Quantum Computer'

# Authors: M. Asaduzzaman, R. G. Jha, B. Sambasivam

## If you use any part of this notebook, we request that you please cite our paper.

### This notebook accompanies the article arXiv: 2311.17991  and can be used to perform the time evolution of the SYK Hamiltonian for a single instance with N=6 Majorana fermions (three qubits) on a real quantum computer. 

### For questions and comments, please email us -- muhammad-asaduzzaman@uiowa.edu, raghav.govind.jha@gmail.com, bsambasi@syr.edu

### Installing and importing packages

In [47]:
!pip install qiskit-experiments

In [48]:
# Importing standard Qiskit libraries
from qiskit import *
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from itertools import combinations
from qiskit import Aer
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile, assemble
from qiskit import IBMQ
from qiskit.providers.aer import QasmSimulator
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.tools.visualization import plot_histogram
from qiskit.tools import job_monitor
from qiskit_experiments.library import LocalReadoutError, CorrelatedReadoutError
from qiskit.result.mitigation.utils import(
    expval_with_stddev,
    str2diag,
    counts_probability_vector
)
from qiskit.providers.fake_provider import FakeHanoiV2
from qiskit.providers.fake_provider import FakeKolkataV2
from qiskit.providers.fake_provider import FakeMumbaiV2
from qiskit.providers.fake_provider import FakeAuckland
from qiskit.providers.fake_provider import FakeSherbrooke
from qiskit.providers.fake_provider import ConfigurableFakeBackend
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import ReadoutError

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

from qiskit.circuit.library import (IGate, XGate, YGate, ZGate,
                                    CXGate, CZGate, ECRGate, iSwapGate)
from qiskit.quantum_info import Operator

from qiskit.circuit import QuantumRegister
from qiskit.dagcircuit import DAGCircuit
from qiskit.transpiler import PassManager
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.transpiler.passes import Optimize1qGatesDecomposition

import numpy as np
import mthree
import datetime

from qiskit_ibm_provider import IBMProvider

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Single qubit Pauli gates
I = IGate()
Z = ZGate()
X = XGate()
Y = YGate()

# 2Q entangling gates
CX = CXGate()
CZ = CZGate()
ECR = ECRGate()
iSwap = iSwapGate()

/opt/conda/lib/python3.10/site-packages/qiskit_ibm_provider/ibm_provider.py:297: UserWarning: Default hub/group/project ibm-q-bnl/bnl-non-c2qa-pro/exploring-fundam not found for the account and is ignored.
  warnings.warn(


### Pauli Twirling/ Randomized Compiling
#### Adapted from https://quantum-enablement.org/posts/2023/2023-02-02-pauli_twirling.html

In [49]:
def generate_pauli_twirling_sets(two_qubit_gate):
    """Generate the Pauli twirling sets for a given 2Q gate
    
    Sets are ordered such that gate[0] and gate[1] are pre-roations
    applied to control and target, respectively.  gate[2] and gate[3]
    are post-rotations for control and target, respectively.
    
    Parameters:
        two_qubit_gate (Gate): Input two-qubit gate
        
    Returns:
        list: List of all twirling gate sets
    """
    # Generate 16 element list of Pauli gates, each repeated 4 times
    operator_list = [I, Z, X, Y]*4
    # This is the target unitary to which our twirled circuit should match
    target_unitary = Operator(two_qubit_gate.to_matrix())
    twirling_sets = []
    
    # For every combination in 16 choose 4 make a circuit and look for equivilence
    for gates in combinations(operator_list, 4):
        # Build a circuit for our twirled 2Q gate
        qc = QuantumCircuit(2)
        qc.append(gates[0], [0])
        qc.append(gates[1], [1])
        qc.append(two_qubit_gate, [0, 1])
        qc.append(gates[2], [0])
        qc.append(gates[3], [1])
        
        norm = np.linalg.norm(Operator.from_circuit(qc)-target_unitary)
        
        phase = None
        # If unitaries match we have a phase of zero
        if abs(norm) < 1e-15:
            phase = 0
        # If unitaries differ by a phase of pi, shift by pi
        elif abs(norm-4) < 1e-15:
            phase = np.pi

        if phase is not None:
            qc.global_phase += phase
            # Verify that our twirled circuit is a valid replacement
            assert Operator.from_circuit(qc) == target_unitary
            twirl_set = (gates, phase)
            # Check that set does not already exist
            if twirl_set not in twirling_sets:
                twirling_sets.append(twirl_set)
            
    return twirling_sets

In [50]:
twirling_groups = {} 

for gate in [CX, CZ, ECR, iSwap]:
    twirl_set = generate_pauli_twirling_sets(gate)
    twirling_groups[gate.name] = twirl_set

In [51]:
class PauliTwirling(TransformationPass):
    """Pauli twirl an input circuit.
    """
    def __init__(self, twirling_gate, seed=None):
        """
        Parameters:
            twirling_gate (str): Which gate to twirl
            seed (int): Seed for RNG, should be < 2e32
        """
        super().__init__()
        # This is the target gate to twirl
        self.twirling_gate = twirling_gate
        # Get the twirling set from the dict we generated above
        # This should be repalced by a cached version in practice
        self.twirling_set = twirling_groups[twirling_gate]
        # Length of the twirling set to bound RNG generation
        self.twirling_len = len(self.twirling_set)
        # Seed the NumPy RNG
        self.rng = np.random.default_rng(seed)

    def run(self, dag):
        """Insert Pauli twirls into input DAG
        
        Parameters:
            dag (DAGCircuit): Input DAG
        
        Returns:
            dag: DAG with twirls added in-place
        """
        for run in dag.collect_runs([self.twirling_gate]):
            for node in run:
                # Generate a random int to specify the twirling gates
                twirl_idx = self.rng.integers(0, self.twirling_len)
                # Get the randomly selected twirling set
                twirl_gates = self.twirling_set[twirl_idx][0]
                twirl_phase = self.twirling_set[twirl_idx][1]
                # Make a small DAG for the twirled circuit we are going to insert
                twirl_dag = DAGCircuit()
                # Add a register of qubits (here always 2Q)
                qreg = QuantumRegister(2)
                twirl_dag.add_qreg(qreg)
                # gate[0] pre-applied to control
                twirl_dag.apply_operation_back(twirl_gates[0], [qreg[0]])
                # gate[1] pre-applied to target
                twirl_dag.apply_operation_back(twirl_gates[1], [qreg[1]])
                # Insert original gate
                twirl_dag.apply_operation_back(node.op, [qreg[0], qreg[1]])
                # gate[2] pre-applied to control
                twirl_dag.apply_operation_back(twirl_gates[2], [qreg[0]])
                # gate[3] pre-applied to target
                twirl_dag.apply_operation_back(twirl_gates[3], [qreg[1]])
                # Add a global phase gate to account for possible phase difference
                twirl_dag.global_phase += twirl_phase
                # Replace the target gate with the twirled version
                dag.substitute_node_with_dag(node, twirl_dag)
        return dag

### Specify circuit parameters, choose backend and qubits to be used

In [52]:
#Number of trotter steps
nt = 8
#Trotter step size. This is just a formality here. Make sure you import the correct file
dt = 1.5

#Choose Backend
#Backend = service.get_backend('ibm_cusco')
#Backend = service.get_backend('ibm_nazca')
Backend = service.get_backend('ibm_kyoto')

#Fake backed=nd
f_Backend = FakeSherbrooke()
#backend=AerSimulator()

#Simulator
t_backend = service.get_backend("ibmq_qasm_simulator")


#Choose nice qubits such that they have high T1, T2 and low readout error. Preferably also low CX/ECR gate time between qubits
q_layout = [4,5,6]

#Specify nunmber of Pauli Twirled circuits for physics and self-mitigation runs
num_phys = 75
num_mit = 75

#Specify the options for the Sampler primitive
options = Options()
options.execution.shots = 2048 # This is the number of shots per circuit
options.resilience_level = 1 # This enables M3 mitigation and DD (dynamical decoupling) error mitigation
options.optimization_level = 1 # This makes sure the circuit remains untouched. In this work, we do all the transpilation before passing it to hardware. This is important for self-mitigation
options.initial_layout = q_layout # Picks the layout of qubits to run the circuit on. While using simulator, make sure the layout of qubits doesn't exceed the number 32

In [53]:
# Generates Pauli-twirled circuits
post_pm = PassManager([PauliTwirling('ecr', seed=54321), Optimize1qGatesDecomposition(Backend.target.operation_names)])
def pt_full_circ(qctrans):
    return(post_pm.run(qctrans))

In [54]:
# Imports trotter circuits for forward and backward time evolution in terms of CX gates + single qubit gates
trot_circ_fwd_cx = QuantumCircuit.from_qasm_file('QC_N6_2H.qasm')
trot_circ_bck_cx = QuantumCircuit.from_qasm_file('QC_N6_2HR.qasm')

In [55]:
# Converts trotter circuits for forward and backward time evolution in terms of ECR gates + single qubit gates
trot_circ_fwd = transpile(trot_circ_fwd_cx, basis_gates = Backend.configuration().basis_gates)
trot_circ_bck = transpile(trot_circ_bck_cx, basis_gates = Backend.configuration().basis_gates)

In [56]:
# Gate counts for forward and backward time evolution to make sure they are the same
[trot_circ_bck.count_ops(), trot_circ_bck.count_ops()]

[OrderedDict([('rz', 53), ('cx', 30), ('sx', 22), ('barrier', 5)]),
 OrderedDict([('rz', 53), ('cx', 30), ('sx', 22), ('barrier', 5)])]

### Building the circuits

In [57]:
# Physics run circuits

full_circ = QuantumCircuit(3)
for i in range(nt):
    full_circ.barrier()
    full_circ = full_circ.compose(trot_circ_fwd)
full_circ.measure_all()
full_circ_trans2 = transpile(full_circ, basis_gates = Backend.configuration().basis_gates, initial_layout = [0,1,2], coupling_map = [[0,1],[1,0],[1,2],[2,1]], optimization_level=2)
full_circ_trans = transpile(full_circ_trans2, basis_gates = Backend.configuration().basis_gates, initial_layout = [0,1,2], coupling_map = [[0,1],[1,0],[1,2],[2,1]], optimization_level=3)

In [58]:
# Self-mitigation run circuits

sm_full_circ = QuantumCircuit(3)
for i in range(int(nt/2)):
    sm_full_circ.barrier()
    sm_full_circ = sm_full_circ.compose(trot_circ_fwd)
for i in range(int(nt/2)):
    sm_full_circ.barrier()
    sm_full_circ = sm_full_circ.compose(trot_circ_bck)
sm_full_circ.measure_all()
sm_full_circ_trans2 = transpile(sm_full_circ, basis_gates = Backend.configuration().basis_gates, initial_layout = [0,1,2], coupling_map = [[0,1],[1,0],[1,2],[2,1]], optimization_level = 2)
sm_full_circ_trans = transpile(sm_full_circ_trans2, basis_gates = Backend.configuration().basis_gates, initial_layout = [0,1,2], coupling_map = [[0,1],[1,0],[1,2],[2,1]], optimization_level = 3)

In [59]:
# Gate counts for full physics and self-mitigation circuits. Important to have same number of two-qubit 
# gates so the mitigation circuits characterize the noise in the physics circuits well
[full_circ_trans.count_ops(),sm_full_circ_trans.count_ops()]

[OrderedDict([('rz', 538),
              ('sx', 330),
              ('cx', 313),
              ('barrier', 41),
              ('x', 24),
              ('measure', 3)]),
 OrderedDict([('rz', 536),
              ('sx', 326),
              ('cx', 313),
              ('barrier', 41),
              ('x', 28),
              ('measure', 3)])]

### Twirl circuits and make run list

In [60]:
# Add twirled circuits to a 2 lists- one for physics and one for self-mitigation
full_circ_list = [pt_full_circ(full_circ_trans) for i in range(num_phys)]
sm_full_circ_list = [pt_full_circ(sm_full_circ_trans) for i in range (num_mit)]

In [61]:
# Another gate count sanity check to make sure #ECR or #CX gates are same
[full_circ_list[0].count_ops(),sm_full_circ_list[69].count_ops()]

[OrderedDict([('rz', 538),
              ('sx', 330),
              ('cx', 313),
              ('barrier', 41),
              ('x', 24),
              ('measure', 3)]),
 OrderedDict([('rz', 536),
              ('sx', 326),
              ('cx', 313),
              ('barrier', 41),
              ('x', 28),
              ('measure', 3)])]

In [62]:
len(sm_full_circ_list)

75

In [63]:
# Circuit list for hardware runs. Concatenated list of physics and mitigation circuits
har_circ_list = []
har_circ_list.extend(full_circ_list)
har_circ_list.extend(sm_full_circ_list)

In [64]:
# Total number of hardware circuits
len(har_circ_list)

150

### Running everything on Fake Backend with results

In [65]:
#Readout-error mitigation runs
num_shots=2048
ro_exp = LocalReadoutError(q_layout)
ro_job = ro_exp.run(f_Backend, shots = num_shots)
mitigator = ro_job.analysis_results(0).value
def Mitigated_Counts(counts):
    mitigated_quasi_probs = mitigator.quasi_probabilities(counts)
    mitigated_stddev = mitigated_quasi_probs._stddev_upper_bound
    mitigated_probs = (mitigated_quasi_probs.nearest_probability_distribution().binary_probabilities())
    mitigated_counts = {label: round(probs*sum(counts.values())) for label, probs in mitigated_probs.items()}
    return(mitigated_counts)

Adding a job from a backend (aer_simulator) that is different than the current backend (<qiskit.providers.fake_provider.backends.sherbrooke.fake_sherbrooke.FakeSherbrooke object at 0x7fa47a8c3a30>). The new backend will be used, but service is not changed if one already exists.


In [66]:
Fback_job_list = execute(har_circ_list, backend = f_Backend, shots = num_shots)

In [67]:
#Compile the counts
Fback_phys_counts_list = [Mitigated_Counts(Fback_job_list.result().get_counts()[i]) for i in range(num_phys)]
Fback_mit_counts_list = [Mitigated_Counts(Fback_job_list.result().get_counts()[i+num_phys]) for i in range(num_mit)]

# Initialize an empty dictionary for the result
Fback_phys_counts_tot = {}
Fback_mit_counts_tot = {}

# Merge dictionaries and calculate the sum of values for common keys
for d in Fback_phys_counts_list:
    for key, value in d.items():
        Fback_phys_counts_tot[key] = Fback_phys_counts_tot.get(key, 0) + value
        
# Merge dictionaries and calculate the sum of values for common keys
for d in Fback_mit_counts_list:
    for key, value in d.items():
        Fback_mit_counts_tot[key] = Fback_mit_counts_tot.get(key, 0) + value

# Output the merged dictionary
print(Fback_phys_counts_tot)
print(Fback_mit_counts_tot)

{'110': 17001, '111': 15713, '001': 19970, '100': 18430, '010': 16717, '101': 20494, '000': 20874, '011': 24406}
{'110': 15603, '111': 15050, '101': 17706, '010': 16146, '100': 17030, '011': 17472, '001': 22724, '000': 31856}


In [68]:
Fback_prob_unmit = ((Fback_phys_counts_tot['000'])/sum(list(Fback_phys_counts_tot.values())))
Fback_ecro_err_p = 1-(Fback_mit_counts_tot['000']/sum(list(Fback_mit_counts_tot.values())))
Fback_prob_mit = (((Fback_phys_counts_tot['000'])/sum(list(Fback_phys_counts_tot.values())))-(1/8)*Fback_ecro_err_p)/(1-Fback_ecro_err_p)

print("========================================================================")
print("Results from", f_Backend)
print("========================================================================")
print("Probability of error from ECR-only mitigation circuits: ", Fback_ecro_err_p)
print("Un-mitigated return probability:", Fback_prob_unmit)
print("Mitigated return probability:", Fback_prob_mit)

Results from <qiskit.providers.fake_provider.backends.sherbrooke.fake_sherbrooke.FakeSherbrooke object at 0x7fa47a8c3a30>
Probability of error from ECR-only mitigation circuits:  0.7925866121481636
Un-mitigated return probability: 0.1358940138667361
Mitigated return probability: 0.1775231952458061


# Running everything

In [69]:
#Check the Backend argument under Circuit Parameters before running this
with Session(service = service, backend = Backend) as session:
    sampler = Sampler(session = session, options = options)
    job = sampler.run(har_circ_list, initial_layout=q_layout)
    print(job.job_id())

base_primitive._run_primitive:INFO:2023-11-22 01:35:49,761: Submitting job using options {'optimization_level': 1, 'resilience_level': 1, 'max_execution_time': None, 'transpilation': {'skip_transpilation': False, 'initial_layout': [4, 5, 6], 'layout_method': None, 'routing_method': None, 'approximation_degree': None, 'optimization_settings': {'level': 1}, 'coupling_map': None, 'basis_gates': None}, 'resilience': {'noise_amplifier': 'TwoQubitAmplifier', 'noise_factors': (1, 3, 5), 'extrapolator': 'LinearExtrapolator', 'level': 1}, 'execution': {'shots': 2048, 'init_qubits': True, 'noise_model': None, 'seed_simulator': None}, 'environment': {'log_level': 'WARNING', 'callback': None, 'job_tags': []}, 'simulator': {'noise_model': None, 'seed_simulator': None, 'coupling_map': None, 'basis_gates': None}}


cnennyfjpbeg008cvc7g


# Results

In [72]:
#Retrieve job using job_id
job_id = "cnennyfjpbeg008cvc7g" # Insert Job tag here from the cell above. 
job_list = service.job(job_id)

#Print out unmitigatd and self-mitigated results
prob_unmit = (1/num_phys)*sum([(job_list.result().quasi_dists[i].nearest_probability_distribution())[0] for i in range(num_phys)])
sm_err_p = 1-(1/num_mit)*sum([(job_list.result().quasi_dists[i+num_phys].nearest_probability_distribution())[0] for i in range(num_mit)])
prob_mit = (prob_unmit - (1/8)*sm_err_p)/(1-sm_err_p)
print("-----------------------------------------------------------------------------------------")
print("The unmitigated return-probability is ", prob_unmit)
print("-----------------------------------------------------------------------------------------")
print("The probability of depolarizing error is ", sm_err_p)
print("-----------------------------------------------------------------------------------------")
print("The self-mitigated return probability is ", prob_mit)
print("-----------------------------------------------------------------------------------------")

#Make list of results from each circuit
ret_prob_list = [(job_list.result().quasi_dists[i].nearest_probability_distribution())[0] for i in range(num_phys)]
sm_err_prob_list = [1-(job_list.result().quasi_dists[i+num_phys].nearest_probability_distribution())[0] for i in range(num_mit)]
full_list = []
full_list.extend([[1.5, 2, 0.00000]])
full_list.extend([ret_prob_list])
full_list.extend([sm_err_prob_list])

#Export the list of results to a file for data analysis and making plots

# Specify the file path where you want to save the nested list
file_path = "6MF_8-1.5.txt"

# Open the file in write mode
with open(file_path, "w") as file:
    # Iterate through the outer list
    for inner_list in full_list:
        # Iterate through the inner list and write each number to the file
        for number in inner_list:
            file.write(str(number) + " ")
        file.write("\n")

# Close the file
file.close()

print(f"Nested list of numbers has been saved to {file_path}")

-----------------------------------------------------------------------------------------
The unmitigated return-probability is  0.11633539766678068
-----------------------------------------------------------------------------------------
The probability of depolarizing error is  0.884053882184524
-----------------------------------------------------------------------------------------
The self-mitigated return probability is  0.050270440300478926
-----------------------------------------------------------------------------------------
Nested list of numbers has been saved to 6MF_8-1.5.txt
